In [1]:
import math
import time
import pandas as pd
import numpy  as np 
from matplotlib import pyplot
from tensorflow import keras
from numpy import array 
from keras.models import Sequential 
from keras.layers import LSTM,GRU,ConvLSTM2D
from keras.layers import RepeatVector
from keras.layers import Dense,Dropout,Flatten,TimeDistributed
from keras.layers import BatchNormalization 
from keras.layers import Bidirectional
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D 
from keras.models import Model 
from keras.layers import Input
from tensorflow.keras.layers import concatenate

In [2]:

def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        if out_end_ix > len(sequences):
            break
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)

    return array(X), array(y)

In [3]:
def read_data(string,n_steps_in,n_steps_out,n_features):
    
    Z = pd.read_csv(string)
    Z=Z.to_numpy()   
    
    X, y = split_sequences(Z, n_steps_in, n_steps_out) 
    
    n_train=int(0.7*len(X))     
    X_train=X[0: n_train,];         y_train = y[0:n_train,]
    X_test =X[n_train: len(X),];    y_test  = y[n_train:len(X),]
    X_train.shape
    X_test.shape 
    X_train[0,]
    
    return X_train,y_train,X_test,y_test 

In [5]:
X_train,y_train,X_test,y_test = read_data("/home/doktormatte/MA_SciComp/ACN_1/Loads/1.csv", 3, 3, 100)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

print(X_train[1473,:,3])
print(y_train[1473])
print('\n')
print(X_train[1474,:,3])
print(y_train[1474])
print('\n')
print(X_train[1475,:,3])
print(y_train[1475])
print('\n')
print(X_train[1476,:,3])
print(y_train[1476])

(73245, 3, 101)
(73245, 3)
(31392, 3, 101)
(31392, 3)
[0. 0. 0.]
[0. 0. 0.]


[0. 0. 0.]
[0. 0. 0.]


[0. 0. 0.]
[0. 0. 0.]


[0. 0. 0.]
[0. 0. 0.]


In [4]:
def fit_model_LSTM(res ,_iter, X_train,y_train,X_test,y_test,
                   n_steps_in,n_steps_out,n_features, nf_1, nf_2, ker_size, 
              po_size,n_n_lstm,dropout,n_epoch,bat_size):
  
    model = Sequential() 
    model.add(LSTM(n_n_lstm,   input_shape=(n_steps_in, n_features)))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(dropout)) 
    model.add(Dense(n_steps_out, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    keras.utils.plot_model(model, show_shapes=True)
    model.fit(X_train, y_train, epochs=n_epoch, batch_size=bat_size,verbose=2)
     
    temp = model.predict(X_test, verbose=2)
    m,n=temp.shape 
    t_target = n_steps_out
       
    yhat=np.zeros((m,t_target))
    y_obs=np.array(y_test[0:m,0:t_target])
    scores1= np.zeros(m)
    
    for i in np.arange(m):  
        for j in np.arange(t_target):  
            if temp[i][j]>=0.5:
                yhat[i][j]=1           
        val=1 - sum(abs(yhat[i,]-y_obs[i,:]))/t_target
        scores1[i]=val       
     
    _mean1 = np.mean(scores1)     
    res[_iter,:]=[nf_1, nf_2, ker_size, po_size, n_n_lstm,dropout,n_epoch,
                  bat_size, _mean1 ]  
    return res 

In [5]:
def run(n_steps_in,n_steps_out,n_features,n_epoch,n_trivals,n_out,
            nf_1,nf_2,ker_size,po_size,n_n_lstm,dropout,bat_size): 
    
    
    n_station=20    
    # string='/home/doktormatte/MA_SciComp/Boulder/'
    string='/home/doktormatte/MA_SciComp/Boulder/Loads/'
    station=[string+'1.csv',string+'2.csv',string+'3.csv',string+'4.csv',string+'5.csv'
             ,string+'6.csv',string+'7.csv',string+'8.csv',string+'9.csv',string+'9.csv'
             ,string+'10.csv',string+'11.csv',string+'12.csv',string+'13.csv',string+'14.csv'
             ,string+'15.csv',string+'16.csv',string+'17.csv',string+'18.csv',string+'19.csv'
             ,string+'20.csv']
  
    res_all=[]
    for s in range(n_station):         
        X_train,y_train,X_test,y_test =read_data(station[s],n_steps_in,n_steps_out,n_features)
        res=np.zeros([n_trivals,n_out])
        X_train.shape
        y_train.shape 
        for _iter in range(n_trivals):               
            res=fit_model_LSTM(res,_iter, X_train,y_train,X_test,y_test, 
                           n_steps_in,n_steps_out,n_features, nf_1, nf_2, 
                           ker_size, po_size,n_n_lstm,dropout,n_epoch,
                           bat_size) 
               
        _mean = np.mean(res[:,-1:],axis=0)
        _std  = np.std(res[:,-1:],axis=0)
        res_all.append([_mean,_std])
        
    temp=[]
    for i in range(n_station):            
        temp.append(res_all[i][0])
        
    accuracy_avg1=np.mean(temp, axis=0)
    accuracy_avg2=np.mean(temp, axis=1)  
    return accuracy_avg1, accuracy_avg2, res_all

In [6]:
def main():
   
    n_steps_in=3  # t-3,t-2,t-1
    n_features=100 
    n_steps_out=3 # num of predicted steps, if n_steps_out =1 or 3 po_size needs to be 1
    n_epoch_global=15
    n_trivals=10
    n_out=9 
    nf_1=32
    nf_2=16
    ker_size=4
    po_size=1
    n_n_lstm=16
    dropout=0.4
    bat_size= 30
    accuracy_avg_1=[]
    accuracy_avg_2=[] 

            
    avg1,avg2,res_all =  run(n_steps_in,n_steps_out,n_features,
                            n_epoch_global,n_trivals,n_out,nf_1,nf_2,ker_size,
                            po_size,n_n_lstm,dropout,bat_size)   
    accuracy_avg_1.append(avg1)
    accuracy_avg_2.append(avg2)
    
    ## output results    
    # print('model: ', model_id)
    # print('n_step out: ', n_steps_out)
    # print('n_epoch,n_trivals, nf_1,nf_2,po_size,n_n_lstm,dropout,bat_size', 
    #       n_epoch_global,n_trivals, nf_1,nf_2,po_size,n_n_lstm,dropout,bat_size)       
    print('accuracy_avg_1: ',accuracy_avg_1)
    print('accuracy_avg_2: ',accuracy_avg_2)
    return res_all

In [7]:
res_all = main()

2022-10-02 17:34:26.728809: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-02 17:34:26.730131: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-02 17:34:26.730271: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-10-02 17:34:26.730379: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-10-02 17:34:26.730484: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

Epoch 1/15
202/202 - 2s - loss: 0.1463 - accuracy: 0.3839 - 2s/epoch - 10ms/step
Epoch 2/15
202/202 - 0s - loss: 0.0180 - accuracy: 0.3087 - 385ms/epoch - 2ms/step
Epoch 3/15
202/202 - 0s - loss: 0.0161 - accuracy: 0.2965 - 388ms/epoch - 2ms/step
Epoch 4/15
202/202 - 0s - loss: 0.0156 - accuracy: 0.2492 - 389ms/epoch - 2ms/step
Epoch 5/15
202/202 - 0s - loss: 0.0157 - accuracy: 0.3079 - 444ms/epoch - 2ms/step
Epoch 6/15
202/202 - 0s - loss: 0.0167 - accuracy: 0.2406 - 422ms/epoch - 2ms/step
Epoch 7/15
202/202 - 0s - loss: 0.0162 - accuracy: 0.2705 - 392ms/epoch - 2ms/step
Epoch 8/15
202/202 - 0s - loss: 0.0142 - accuracy: 0.2957 - 398ms/epoch - 2ms/step
Epoch 9/15
202/202 - 0s - loss: 0.0149 - accuracy: 0.3132 - 391ms/epoch - 2ms/step
Epoch 10/15
202/202 - 0s - loss: 0.0159 - accuracy: 0.2255 - 406ms/epoch - 2ms/step
Epoch 11/15
202/202 - 0s - loss: 0.0135 - accuracy: 0.2429 - 479ms/epoch - 2ms/step
Epoch 12/15
202/202 - 0s - loss: 0.0148 - accuracy: 0.3691 - 475ms/epoch - 2ms/step
Epo

In [7]:
string='/home/doktormatte/MA_SciComp/Boulder/'
station=[string+'1.csv',string+'2.csv',string+'3.csv',string+'4.csv',string+'5.csv',string+'6.csv',string+'7.csv',string+'8.csv',string+'9.csv']

In [8]:
model_id = 'LSTM'
t_win = 18
n_steps_in = 3
n_steps_out = 6
n_features = 100
n_trivals=10
n_out=9 
nf_1=32
nf_2=16
ker_size=4
po_size=2
n_n_lstm=16
dropout=0.4
bat_size= 30
n_epoch=50

In [9]:
#filename = '/home/doktormatte/Dropbox/Dokumente/Studium/MA_SciComp/Code/Multistep-Electric-Vehicle-Charging-Station-Occupancy-Prediction--main/DL variants/data_chg_all_feature_short.csv'
filename = '/home/doktormatte/MA_SciComp/Boulder/try.csv'
#data_chg_all_feature_try
X_train,y_train,X_test,y_test =read_data(filename,model_id, t_win, n_steps_in,n_steps_out,n_features)

print(X_train.shape)
print(y_train.shape)

(3, 100)
(6,)


(6042, 3, 100)
(6042, 6)


In [11]:
res = np.zeros([n_trivals,n_out])
_iter = 0

In [12]:
res=fit_model_LSTM(res,_iter, X_train,y_train,X_test,y_test ,t_win, 
                           n_steps_in,n_steps_out,n_features, nf_1, nf_2, 
                           ker_size, po_size,n_n_lstm,dropout,n_epoch,
                           bat_size) 

2022-09-21 11:56:59.307805: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-21 11:56:59.308651: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-21 11:56:59.308747: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-09-21 11:56:59.308812: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-09-21 11:56:59.308876: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
Epoch 1/50
202/202 - 2s - loss: 0.4940 - accuracy: 0.1140 - 2s/epoch - 10ms/step
Epoch 2/50
202/202 - 0s - loss: 0.2377 - accuracy: 0.1352 - 386ms/epoch - 2ms/step
Epoch 3/50
202/202 - 0s - loss: 0.2036 - accuracy: 0.1696 - 384ms/epoch - 2ms/step
Epoch 4/50
202/202 - 0s - loss: 0.1925 - accuracy: 0.1802 - 374ms/epoch - 2ms/step
Epoch 5/50
202/202 - 0s - loss: 0.1830 - accuracy: 0.1844 - 384ms/epoch - 2ms/step
Epoch 6/50
202/202 - 0s - loss: 0.1771 - accuracy: 0.1912 - 382ms/epoch - 2ms/step
Epoch 7/50
202/202 - 0s - loss: 0.1758 - accuracy: 0.1946 - 379ms/epoch - 2ms/step
Epoch 8/50
202/202 - 0s - loss: 0.1750 - accuracy: 0.1965 - 403ms/epoch - 2ms/step
Epoch 9/50
202/202 - 0s - loss: 0.1757 - accuracy: 0.1958 - 408ms/epoch - 2ms/step
Epoch 10/50
202/202 - 0s - loss: 0.1735 - accuracy: 0.1984 - 408ms/epoch - 2ms/step
Epoch 11/